In [1]:
import numpy as np
import pandas as pd

In [55]:
enrollment_data = pd.read_excel("Raw Data/Anon Enrollment Data.xlsx")

unique_students = enrollment_data["UUN"].unique()
unique_years = enrollment_data["Year Of Programme"].unique()
unique_degrees = enrollment_data["Programme Of Study Sought Title"].unique()
unique_schools = enrollment_data["Programme School Name"].unique()
unique_years_taken = enrollment_data["Normal Year Taken"].unique()
unique_course_codes = enrollment_data['Course Code'].unique()
unique_courses = enrollment_data['Course Name'].unique()

In [61]:
enrollment_cols = list(enrollment_data.columns)

In [60]:
enrollment_data.loc[enrollment_data["Normal Year Taken"]=="P"].shape[0]

3167

In [77]:
# for i in range(5)
# enrollment_data.loc[enrollment_data["Year Of Programme"] == 1]
# som_only = enrollment_data.loc[enrollment_data["Programme School Name"] == "School of Mathematics"]

# unique_degrees_som = som_only["Programme Of Study Sought Title"].unique()
# list(unique_degrees_som)


# som_only = enrollment_data.loc[(enrollment_data["Programme Of Study Sought Title"] == 'Algebra and Number Theory') &
#                                (enrollment_data["Programme School Name"] == 'School of Mathematics')]

enrollment_data.loc[enrollment_data["UUN"] == "C1335471"]

# som_only

,UUN,Year Of Programme,Programme Of Study Sought Title,Programme School Name,Normal Year Taken,Course Code,Course Name
242,C1335471,2,Algebra and Number Theory,School of Mathematics,5,MATH11120,Algebraic Geometry
7784,C1335471,2,Algebra and Number Theory,School of Mathematics,5,MATH11053,Introduction to Lie Groups
